In [62]:
import pandas as pd
import numpy

In [136]:
# definir SelectFiles qui retourne une lsite de dataframes + renommer les colonnes 

def select_files():
    df = pd.read_excel("./fot10_copie.xlsx",usecols="B,F,I,J,Q,s:w").copy()[21:] # Ouvrir le fichier et ne regarder que les colonnes interessantes + a partir de ligne 21
    pd.set_option('display.max_rows', None)

    df.rename(columns={
        'Unnamed: 1': 'N° Cote',  # Remplacez 'NouveauNom1' par le nom que vous souhaitez
        'Unnamed: 5': 'Valeur nominale',  # Remplacez 'NouveauNom2' par le nom que vous souhaitez
        'Unnamed: 8': 'Limite Inf',  # etc.
        'Unnamed: 9': 'Limite Sup',
        'Unnamed: 16': 'Empreinte',
        'Unnamed: 18': 'Mesure1',
        'Unnamed: 19': 'Mesure2',
        'Unnamed: 20': 'Mesure3',
        'Unnamed: 21': 'Mesure4',
        'Unnamed: 22': 'Mesure5'
    }, inplace=True)
    
    return df


In [143]:
def replace_all(df): # remplace toutes les aleurs autre que int ou float par 0
        for column in df.columns:
                df[column] = pd.to_numeric(df[column], errors='coerce') # Dans toutes les colones on converti chaque valeur en float, si erreur remplacer nan
        df.fillna(0, inplace=True) # remplace nan par 0 partout
        
        return df

def keep_pairs_lines(df):
        indices = df.index.tolist()
        indices_to_keep = list(indices)  # Utilisation d'un set pour éviter les doublons
        
        # Ajout de l'indice de la ligne suivante pour chaque ligne conforme
        for index in indices:
                next_index = index + 1
                if next_index in df.index:  # Assurez-vous que l'indice suivant est dans le DataFrame original
                        indices_to_keep.append(next_index)

        # Création du nouveau DataFrame avec les lignes sélectionnées
        
        df = df.loc[indices_to_keep]
        
        # Tri du DataFrame selon l'ordre initial des indices
        dc = df.sort_index()
        return dc

def clean_data(df):
        
        dfcotes = replace_all(df)
        
        dfcotes = df[df['N° Cote'] > 10] # ne conserve que les lignes ou la valeur de la colonne est supérieure à 10 pour éliminer tous les espaces 
        
        dc = keep_pairs_lines(dfcotes)

        dc.reset_index(drop=True, inplace=True) 
        

        return dc

df = select_files()

dc = clean_data(df)



None


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [97]:

def check_row(row, nominal, lim_inf, lim_sup):
    return all([
        (row[f'Mesure{i}'] == 0) or (row[f'Mesure{i}'] < (nominal + lim_sup) and row[f'Mesure{i}'] > (nominal + lim_inf)) 
        for i in range(1, 6)
    ])

# Création d'une liste pour les indices des lignes à conserver
indices_to_keep = []
indice_to_move = []
# Parcours du DataFrame en sautant une ligne à chaque fois (pour traiter les paires)
for index in range(0, len(dc), 2):
    if index + 1 in dc.index:  # Vérifier que la paire existe
        row1 = dc.loc[index]
        row2 = dc.loc[index + 1]
        nominal, lim_inf, lim_sup = row1['Valeur nominale'], row1['Limite Inf'], row1['Limite Sup']

        # Conserver la paire si les deux lignes répondent aux critères
        if check_row(row1, nominal, lim_inf, lim_sup) and check_row(row2, nominal, lim_inf, lim_sup):
            indices_to_keep.extend([index, index + 1])
        else:
            indice_to_move.extend([index, index + 1])
        

# Filtrer le DataFrame pour ne garder que les lignes avec les indices sélectionnés
ds = dc.loc[indices_to_keep] # conformes
dm= dc.loc[indice_to_move] # non conformes
# Affiche toutes les côtes conformes
nonConformes = len(dm)/2
conformes = len(ds)/2
print("Nb de non conformes : " + str(nonConformes))
print("Nb de  conformes : "+ str(conformes))

Nb de non conformes : 144.0
Nb de  conformes : 163.0
